In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import torch
from torchvision.datasets import CIFAR100
from torchvision.transforms import Normalize, ToTensor, Lambda, Compose
import torch.nn.functional as F
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim
from ipywidgets import interact, fixed
import ipywidgets as widgets
import cifar_names as cn

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
image_transforms = Compose([
    ToTensor(), 
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

label_transforms = Compose([
    Lambda(lambda label: (label, cn.SUPERCLASS_MAPPING[label])),
    Lambda(lambda lsl: (torch.tensor(lsl[0]), torch.tensor(lsl[1])))
])

In [ ]:
cifar100_path = Path("data/")
cifar100_train, cifar100_test = [
    CIFAR100(
        cifar100_path, 
        train=is_train, 
        download=True,
        transform=image_transforms,
        target_transform=label_transforms
    )
    for is_train in [True, False]
]

In [ ]:
def show_example(i: int):
    img, (c, sc)  = cifar100_train[i]
    plt.imshow(img.permute(1,2,0))
    plt.title(f"class: {cn.CIFAR100_LABELS_LIST[c]}, superclass: {cn.SUPERCLASS_LIST[sc]}")
    plt.show()
    
interact(show_example, i=widgets.IntSlider(max=len(cifar100_train)))

In [ ]:
loader_train = DataLoader(cifar100_train, batch_size=32,shuffle=True)
loader_test = DataLoader(cifar100_test, batch_size=32)
iter_train = iter(loader_train)
iter_test = iter(loader_test)

In [ ]:
x, (y_c, y_s) = next(iter_train)